<a href="https://colab.research.google.com/github/achrisk/Dissertation/blob/main/Copy_of_within_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/achrisk/Dissertation.git

Cloning into 'Dissertation'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), done.
Checking out files: 100% (24/24), done.


In [ ]:
import scipy.io as spio
import numpy as np
import pandas as pd
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from Dissertation.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

In [ ]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [ ]:
K.clear_session()

X_tr = np.empty([80, 12, 4096])
result=[]

h_cut = [12, 18, 30] # [only alpha, alpha + low beta, alpha + beta]
drop_out = [0.25, 0.5]
k_len = [64, 128, 256, 512]
n_epochs = 500
model_name = 'EEGNet'
kernels, chans, samples = 1, 12, 1536

outfname = 'accuray_epochs' + str(n_epochs) + '_filter_' + str(h_cut) + '_drop_' + str(drop_out) + '_model_' + str(model_name) + '_patient_data.npy'

nsub = 8
nfilt = len(h_cut)
ndrop = len(drop_out)
nkl = len(k_len)
acc_sub = np.zeros([nsub, nfilt,ndrop,nkl])
hyper_parameter_matrix = np.zeros([nsub, nfilt, ndrop, nkl, 5])

for sub_idx, x in enumerate(range(1,nsub+1)):
    for h_indx, h in enumerate(h_cut):
        fName = 'Dissertation/EEG data/parsed_P0' + str(x) + 'T.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_tr = mat['RawEEGData']
        y_tr = mat['Labels']
        y_tr = y_tr.flatten() 

        print(np.shape(r_X_tr))
        print(np.shape(y_tr))

        for t in range(r_X_tr.shape[0]):
            tril = r_X_tr[t,:,:]
            #tril = tril.transpose()
            tril_filtered = butter_bandpass_filter(tril, lowcut=8, highcut=h, fs=250, order=4)
            # tril_filtered = tril_filtered.transpose()
            X_tr[t,:,:] = tril_filtered 

        # split data of each subject in training and validation
        X_train = X_tr[0:50,:,2048:3584]
        Y_train = y_tr[0:50]
        X_val = X_tr[50:70,:,2048:3584]
        Y_val = y_tr[50:70]
        X_test = X_tr[70:,:,2048:3584]
        Y_test = y_tr[70:]

        print(np.shape(X_train))
        print(np.shape(Y_train))
        print(np.shape(X_val))
        print(np.shape(Y_val))
        print(np.shape(X_test))
        print(np.shape(Y_test))

        # convert labels to one-hot encodings.
        Y_train = np_utils.to_categorical(Y_train-1, num_classes=2)
        Y_val = np_utils.to_categorical(Y_val-1, num_classes=2)
        Y_test = np_utils.to_categorical(Y_test-1, num_classes=2)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val = X_val.reshape(X_val.shape[0], kernels, chans, samples)
        X_test = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'test samples')

        for id_d, d in enumerate(drop_out):
            for id_kl, kl in enumerate(k_len):
              print(id_kl, id_d)
              # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
              # model configurations may do better, but this is a good starting point)
              model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples,
                             dropoutRate = d, kernLength = kl, F1 = 8,D = 2, F2 = 16,
                             norm_rate = 0.25, dropoutType = 'Dropout')
              
              # model = ShallowConvNet(nb_classes = 2, Chans = chans, Samples = samples, dropoutRate = d)
              
              # model = DeepConvNet(nb_classes, Chans = chans, Samples = samples, dropoutRate = d)
              

              # compile the model and set the optimizers
              model.compile(loss='categorical_crossentropy', optimizer='adam', 
                            metrics = ['accuracy'])

              # count number of parameters in the model
              numParams    = model.count_params() 

              # set a valid path for your system to record model checkpoints
              checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

              # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
              # the weights all to be 1
              class_weights = {0:1, 1:1}

              history = model.fit(X_train, Y_train, batch_size = 16, epochs = n_epochs, 
                                  verbose = 2, validation_data=(X_val, Y_val),
                                  callbacks=[checkpointer], class_weight = class_weights)

              print('\n# Evaluate on test data')


              results = model.evaluate(X_test, Y_test)
              print('test loss, test acc:', results)

              print(results)
              acc_sub[sub_idx,h_indx,id_d, id_kl] = results[1]
              hyper_parameter_matrix[sub_idx,h_indx,id_d, id_kl, :] = [x, h, d, kl, results[1]]

              K.clear_session()
              
np.save(outfname, hyper_parameter_matrix)


Streaming output truncated to the last 5000 lines.
Epoch 00262: val_loss did not improve from 0.31649
4/4 - 0s - loss: 0.0469 - accuracy: 1.0000 - val_loss: 0.5518 - val_accuracy: 0.7000
Epoch 263/500

Epoch 00263: val_loss did not improve from 0.31649
4/4 - 0s - loss: 0.0713 - accuracy: 1.0000 - val_loss: 0.5569 - val_accuracy: 0.7000
Epoch 264/500

Epoch 00264: val_loss did not improve from 0.31649
4/4 - 0s - loss: 0.0710 - accuracy: 1.0000 - val_loss: 0.5371 - val_accuracy: 0.7000
Epoch 265/500

Epoch 00265: val_loss did not improve from 0.31649
4/4 - 0s - loss: 0.0662 - accuracy: 1.0000 - val_loss: 0.5339 - val_accuracy: 0.6500
Epoch 266/500

Epoch 00266: val_loss did not improve from 0.31649
4/4 - 0s - loss: 0.0576 - accuracy: 1.0000 - val_loss: 0.5376 - val_accuracy: 0.6500
Epoch 267/500

Epoch 00267: val_loss did not improve from 0.31649
4/4 - 0s - loss: 0.0377 - accuracy: 1.0000 - val_loss: 0.5084 - val_accuracy: 0.6500
Epoch 268/500

Epoch 00268: val_loss did not improve from 

In [ ]:
!ls

'accuray_epochs500_filter_[12]_drop_[0.25]_model_DeepConvNet_patient_data.npy'
'accuray_epochs500_filter_[12]_drop_[0.25]_model_EEGNet_patient_data.npy'
'accuray_epochs500_filter_[12]_drop_[0.25]_model_ShallowConvNet_patient_data.npy'
 Dissertation
 sample_data


In [ ]:
K.clear_session()

X_tr = np.empty([80, 12, 4096])
result=[]

h_cut = [12, 18, 30] # [only alpha, alpha + low beta, alpha + beta]
drop_out = [0.25, 0.5]
n_epochs = 500
model_name = 'ShallowConvNet'  # ShallowConvNet or DeepConvNet
kernels, chans, samples = 1, 12, 1536

outfname = 'accuray_epochs' + str(n_epochs) + '_filter_' + str(h_cut) + '_drop_' + str(drop_out) + '_model_' + str(model_name) + '_patient_data.npy'

nsub = 8
nfilt = len(h_cut)
ndrop = len(drop_out)
acc_sub = np.zeros([nsub, nfilt,ndrop,nkl])
hyper_parameter_matrix = np.zeros([nsub, nfilt, ndrop, 4])

for sub_idx, x in enumerate(range(1,nsub+1)):
    for h_indx, h in enumerate(h_cut):
        fName = 'Dissertation/EEG data/parsed_P0' + str(x) + 'T.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_tr = mat['RawEEGData']
        y_tr = mat['Labels']
        y_tr = y_tr.flatten() 

        print(np.shape(r_X_tr))
        print(np.shape(y_tr))

        for t in range(r_X_tr.shape[0]):
            tril = r_X_tr[t,:,:]
            #tril = tril.transpose()
            tril_filtered = butter_bandpass_filter(tril, lowcut=8, highcut=h, fs=250, order=4)
            # tril_filtered = tril_filtered.transpose()
            X_tr[t,:,:] = tril_filtered 

        # split data of each subject in training and validation
        X_train = X_tr[0:50,:,2048:3584]
        Y_train = y_tr[0:50]
        X_val = X_tr[50:70,:,2048:3584]
        Y_val = y_tr[50:70]
        X_test = X_tr[70:,:,2048:3584]
        Y_test = y_tr[70:]

        print(np.shape(X_train))
        print(np.shape(Y_train))
        print(np.shape(X_val))
        print(np.shape(Y_val))
        print(np.shape(X_test))
        print(np.shape(Y_test))

        # convert labels to one-hot encodings.
        Y_train = np_utils.to_categorical(Y_train-1, num_classes=2)
        Y_val = np_utils.to_categorical(Y_val-1, num_classes=2)
        Y_test = np_utils.to_categorical(Y_test-1, num_classes=2)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val = X_val.reshape(X_val.shape[0], kernels, chans, samples)
        X_test = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'test samples')

        for id_d, d in enumerate(drop_out):
          print(id_kl, id_d)
          # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
          # model configurations may do better, but this is a good starting point)
          
          if model_name == 'DeepConvNet':
            model = DeepConvNet(nb_classes=2, Chans = chans, Samples = samples, dropoutRate = d)
          elif model_name == 'ShallowConvNet':
            model = ShallowConvNet(nb_classes = 2, Chans = chans, Samples = samples, dropoutRate = d)
          else:
            raise ValueError('Model name wrong!')
          

          # compile the model and set the optimizers
          model.compile(loss='categorical_crossentropy', optimizer='adam', 
                        metrics = ['accuracy'])

          # count number of parameters in the model
          numParams    = model.count_params() 

          # set a valid path for your system to record model checkpoints
          checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

          # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
          # the weights all to be 1
          class_weights = {0:1, 1:1}

          history = model.fit(X_train, Y_train, batch_size = 16, epochs = n_epochs, 
                              verbose = 2, validation_data=(X_val, Y_val),
                              callbacks=[checkpointer], class_weight = class_weights)

          print('\n# Evaluate on test data')


          results = model.evaluate(X_test, Y_test)
          print('test loss, test acc:', results)

          print(results)
          acc_sub[sub_idx,h_indx,id_d] = results[1]
          hyper_parameter_matrix[sub_idx,h_indx,id_d, :] = [x, h, d, results[1]]

          K.clear_session()
              
np.save(outfname, hyper_parameter_matrix)


Dissertation/EEG data/parsed_P01T.mat
(80, 12, 4096)
(80,)
(50, 12, 1536)
(50,)
(20, 12, 1536)
(20,)
(10, 12, 1536)
(10,)
X_train shape: (50, 1, 12, 1536)
50 train samples
20 val samples
X_train shape: (10, 1, 12, 1536)
10 test samples
0 0
Epoch 1/500

Epoch 00001: val_loss improved from inf to 2.20251, saving model to /tmp/checkpoint.h5
4/4 - 0s - loss: 1.1122 - accuracy: 0.5800 - val_loss: 2.2025 - val_accuracy: 0.3000
Epoch 2/500

Epoch 00002: val_loss improved from 2.20251 to 1.64431, saving model to /tmp/checkpoint.h5
4/4 - 0s - loss: 0.8104 - accuracy: 0.7400 - val_loss: 1.6443 - val_accuracy: 0.5000
Epoch 3/500

Epoch 00003: val_loss did not improve from 1.64431
4/4 - 0s - loss: 0.3974 - accuracy: 0.8600 - val_loss: 1.9149 - val_accuracy: 0.5000
Epoch 4/500

Epoch 00004: val_loss did not improve from 1.64431
4/4 - 0s - loss: 0.0421 - accuracy: 1.0000 - val_loss: 2.9386 - val_accuracy: 0.3500
Epoch 5/500

Epoch 00005: val_loss did not improve from 1.64431
4/4 - 0s - loss: 0.1048 

In [ ]:
ind = np.unravel_index(np.argmax(acc_sub, axis=None), acc_sub.shape)
acc_sub
hyper_parameter_matrix
hyper_parameter_matrix[ind]
reduced_acc_sub = np.mean(acc_sub, axis=0, keepdims=True)
ind_reduced = np.unravel_index(np.argmax(reduced_acc_sub, axis=None), reduced_acc_sub.shape)
print('Index is: {}'.format(ind_reduced[-1]))
print('Hyperparameters are: {}'.format(hyper_parameter_matrix[ind_reduced]))
print(hyper_parameter_matrix[ind_reduced])
print(acc_sub)

Index is: 0
Hyperparameters are: [ 1.  24.   0.5 32. ]
[ 1.  24.   0.5 32. ]
[[[[0.60000002 0.69999999 0.55000001]
   [0.60000002 0.64999998 0.55000001]
   [0.69999999 0.64999998 0.60000002]]]


 [[[0.69999999 0.85000002 0.64999998]
   [0.85000002 0.80000001 0.94999999]
   [0.69999999 0.64999998 0.94999999]]]


 [[[0.         0.         0.        ]
   [0.         0.05       0.05      ]
   [0.05       0.05       0.        ]]]


 [[[0.80000001 0.69999999 0.80000001]
   [0.85000002 0.80000001 0.60000002]
   [0.75       0.80000001 0.69999999]]]


 [[[0.44999999 0.40000001 0.34999999]
   [0.5        0.34999999 0.40000001]
   [0.40000001 0.5        0.34999999]]]


 [[[0.60000002 0.64999998 0.64999998]
   [0.69999999 0.60000002 0.64999998]
   [0.55000001 0.5        0.60000002]]]


 [[[0.5        0.60000002 0.44999999]
   [0.5        0.5        0.5       ]
   [0.5        0.5        0.5       ]]]


 [[[0.69999999 0.75       0.69999999]
   [0.80000001 0.80000001 0.60000002]
   [0.85000002 0.6499